<div style="text-align: right">INFO 6105 Data Science Eng Methods and Tools</div>
<div style="text-align: right">Yanhong Chen 0018184656</div>

# Homework

Use the methodology in this notebook to build a statistical language translator, *from your language to English*. So, from Hindi or Chinese to English. Teams of **3** students. You *have* to use a Hidden Markov Model and `pomegranate` as your HMM library, to ensure all student teams start from the same baseline. Start from a Most Frequent Word (BOW) translation baseline, then move on to a Hidden Markov Model to improve translation. How much can you improve it by? The translation engine with the best accuracy ***improvement***, per language, will be presented in class.

ADJ 形容词
ADP 介词
ADV 副词
CONJ 连词
DET 限定词
NOUN 名词
NUM 数词
PRON 代词
PRT 缩写
VERB 动词
X 其他

In [3]:
### pip install jieba ### "Jieba" (Chinese for "to stutter") Chinese text segmentation
### pip install paddlepaddle-tiny==1.6.1

In [1]:
### pip install nltk ### NLTK is a leading platform for building Python programs to work with human language data
### nltk.download('punkt')

In [4]:
### pip install googletrans ### googletrans API

In [1]:
### pip install youdao-tr-free ### youdao translate API

In [ ]:
### nltk.download('stopwords')

In [26]:
### nltk.download('averaged_perceptron_tagger')

In [1]:
### pip install pinyin

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict, namedtuple, OrderedDict
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution
import os
from io import BytesIO
from itertools import chain

import random
import jieba
import jieba.posseg as pseg
jieba.enable_paddle()
import nltk
import pinyin
import pinyin.cedict
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
from nltk.tokenize import RegexpTokenizer
stop_words = set(stopwords.words('english')) 
from googletrans import Translator
from youdao_tr import youdao_tr

Paddle enabled successfully......


In [5]:
def read_data_for_sentence(filename):
    """Read tagged sentence data"""
    with open(filename, mode='r', encoding='UTF-8-sig') as f:
        sentence_lines = [l.split("\n") for l in f.read().split("\n\n")]
    return sentence_lines
originSentences = read_data_for_sentence("Translate_Chinese_To_English/Translate_Chinese_To_English.txt")

In [6]:
originalEnglishSentences = originSentences[0][0:len(originSentences[0])-1:2]
### print(originalEnglishSentences)
len(originalEnglishSentences)

9757

In [7]:
originalChineseSentences = originSentences[0][1:len(originSentences[0])-1:2]
### print(originalChineseSentences)
len(originalChineseSentences)

9757

n 普通名词  NOUN
f 方位名词  NOUN
s 处所名词  NOUN
t 时间     NOUN
nr 人名    NOUN
ns 地名    NOUN
nt 机构名  NOUN
nw 作品名  NOUN
nz 其他专名 NOUN
v 普通动词 VERB
vd 动副词 VERB
vn 名动词 VERB 
a 形容词  ADJ
ad 副形词 ADJ
an 名形词 ADJ
d 副词   ADV
m 数量词 NUM
q 量词  NUM
r 代词  PRON 
p 介词  ADP
c 连词  CONJ
u 助词  X
xc 其他虚词  X
w 标点符号  X
PER 人名  NOUN 
LOC 地名  NOUN
ORG 机构名  NOUN
TIME 时间  NOUN

In [8]:
transferChineseTags = {'n':'NOUN','f':'NOUN','s':'NOUN','t':'NOUN','nr':'NOUN','ns':'NOUN','nt':'NOUN','nw':'NOUN',
                     'nz':'NOUN','v':'VERB','vd':'VERB','vn':'VERB','a':'ADJ','ad':'ADJ','an':'ADJ','d':'ADV',
                     'm':'NUM','q':'NUM','r':'PRON','p':'ADP','c':'CONJ','u':'X','xc':'X','w':'X','PER':'NOUN',
                     'LOC':'NOUN','ORG':'NOUN','TIME':'NOUN'}
transferChineseTags['m']

'NUM'

In [9]:
separatedChineseSentences = []
for str in originalChineseSentences:
    words = zip(*pseg.cut(str,use_paddle=True))
    w = list(words)
    words = list(w[0])
    tags = []
    for tag in w[1]:
        tags.append(transferChineseTags[tag])
    separatedChineseSentences.append([words,tags])
print("Three sample of separated Chinese Sentences: {}%".format(separatedChineseSentences[:3]))
### list : first part of list [:3]

Three sample of separated Chinese Sentences: [[['海湾', '里', '平静', '海水', '的', '迷人', '景色'], ['NOUN', 'NOUN', 'ADJ', 'NOUN', 'X', 'ADJ', 'NOUN']], [['计算机病毒', '的', '一种', '主要', '特性'], ['NOUN', 'X', 'NUM', 'ADJ', 'NOUN']], [['一种', '致命', '的', '疾病'], ['NUM', 'VERB', 'X', 'NOUN']]]%


In [10]:
transferEnglishTags = {'CC':'CONJ','CD':'NUM','DT':'DET','EX':'X','FW':'X','IN':'ADP','JJ':'ADJ','JJR':'ADJ',
                       'JJS':'ADJ','LS':'X','MD':'VWRB','NN':'NOUM','CC':'CONJ','CD':'NUM','DT':'DET','EX':'x',
                       'FW':'x','IN':'ADP','JJ':'ADJ','JJR':'ADJ','JJS':'ADJ','LS':'x','MD':'VERB','NN':'NOUN',
                       'NNS':'NOUN','NNP':'NOUN','NNPS':'NOUN','PDT':'DET','POS':'PRON','PRP':'PRON','PRP$':'PRON',
                       'RB':'ADV','RBR':'ADV','RBS':'ADV','RP':'x','TO':'x','UH':'x','VB':'VERB','VBD':'VERB',
                       'VBG':'VERB','VBN':'VERB','VBP':'VERB','VBZ':'VERB','WDT':'DET','WP':'PRON','WP$':'PRON',
                        'WRB':'ADV',':' : 'X'} 
transferEnglishTags['VBG']

'VERB'

In [11]:
import string
separatedEnglishSentences = []
for str in originalEnglishSentences:
    wordsList = nltk.word_tokenize(str)
    wordsList = [w for w in wordsList if not (w in stop_words or w in string.punctuation)] 
    words = zip(*nltk.pos_tag(wordsList))
    w = list(words)
    words = list(w[0])
    tags = []
    for tag in w[1]:
        tags.append(transferEnglishTags[tag])
    separatedEnglishSentences.append([words, tags])
print("Three sample of separated English Sentences: {}%".format(separatedEnglishSentences[:3]))

Three sample of separated English Sentences: [[['A', 'magical', 'view', 'calm', 'waters', 'bay'], ['DET', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'VERB']], [['A', 'main', 'feature', 'computer', 'virus'], ['DET', 'ADJ', 'NOUN', 'NOUN', 'NOUN']], [['A', 'malignant', 'disease'], ['DET', 'ADJ', 'NOUN']]]%


In [12]:
ListPair = zip(separatedChineseSentences,separatedEnglishSentences)
list(ListPair)[1:3]

[([['计算机病毒', '的', '一种', '主要', '特性'], ['NOUN', 'X', 'NUM', 'ADJ', 'NOUN']],
  [['A', 'main', 'feature', 'computer', 'virus'],
   ['DET', 'ADJ', 'NOUN', 'NOUN', 'NOUN']]),
 ([['一种', '致命', '的', '疾病'], ['NUM', 'VERB', 'X', 'NOUN']],
  [['A', 'malignant', 'disease'], ['DET', 'ADJ', 'NOUN']])]

In [13]:
from googletrans import Translator
translator = Translator()
a = translator.translate('안녕하세요.')
a.text

'Hello.'

In [4]:
pinyin.cedict.translate_word('我们')

['we', 'us', 'ourselves', 'our']

In [5]:
pinyin.cedict.translate_word('球赛')

['sports match', 'ballgame', 'CL:場|场[chang3]']

In [14]:
ListPair = zip(separatedChineseSentences,separatedEnglishSentences)
listPair = list(ListPair)
Sentence = namedtuple("Sentence", "chineseWords equivalentEnglish")
orderedDict = OrderedDict(((idx, Sentence(tuple(val[0]), tuple(val[1]))) for idx, val in enumerate(listPair) if val[0]))
orderedDict

OrderedDict([(0,
              Sentence(chineseWords=(['海湾', '里', '平静', '海水', '的', '迷人', '景色'], ['NOUN', 'NOUN', 'ADJ', 'NOUN', 'X', 'ADJ', 'NOUN']), equivalentEnglish=(['A', 'magical', 'view', 'calm', 'waters', 'bay'], ['DET', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'VERB']))),
             (1,
              Sentence(chineseWords=(['计算机病毒', '的', '一种', '主要', '特性'], ['NOUN', 'X', 'NUM', 'ADJ', 'NOUN']), equivalentEnglish=(['A', 'main', 'feature', 'computer', 'virus'], ['DET', 'ADJ', 'NOUN', 'NOUN', 'NOUN']))),
             (2,
              Sentence(chineseWords=(['一种', '致命', '的', '疾病'], ['NUM', 'VERB', 'X', 'NOUN']), equivalentEnglish=(['A', 'malignant', 'disease'], ['DET', 'ADJ', 'NOUN']))),
             (3,
              Sentence(chineseWords=(['人生', '只有', '一回', '死'], ['NOUN', 'VERB', 'NUM', 'VERB']), equivalentEnglish=(['A', 'man', 'die'], ['DET', 'NOUN', 'NOUN']))),
             (4,
              Sentence(chineseWords=(['出去', '吃', '晚餐', '如何'], ['VERB', 'VERB', 'NOUN', 'PRON']), equivalentEng

In [15]:
def read_data_return_orderedDict(filename):
    """Read tagged sentence data"""
    with open(filename, mode='r', encoding='UTF-8-sig') as f:
        sentence_lines = [l.split("\n") for l in f.read().split("\n\n")]
    ### originalEnglishSentences = sentence_lines[0][0:len(sentence_lines[0])-1:2]
    ### originalChineseSentences = sentence_lines[0][1:len(sentence_lines[0])-1:2]  
    originalEnglishSentences = sentence_lines[0][0:300:2]
    originalChineseSentences = sentence_lines[0][1:300:2]  
    separatedChineseSentences = []
    for str1 in originalChineseSentences:
        seg_list = jieba.cut(str1, cut_all=False)
        separatedChineseSentences.append(list(seg_list))
    separatedEnglishSentences = []
    for str2 in originalEnglishSentences:
        tokens = nltk.word_tokenize(str2)
        separatedEnglishSentences.append(tokens)
    ListPair = zip(separatedChineseSentences,separatedEnglishSentences)
    listPair = list(ListPair)
    Sentence = namedtuple("Sentence", "chineseWords equivalentEnglish ")
    orderedDict = OrderedDict(((idx, Sentence(tuple(val[0]), tuple(val[1]))) for idx, val in enumerate(listPair) if val[0]))
    orderedDict
    return orderedDict

In [16]:
orderedDict = read_data_return_orderedDict("Translate_Chinese_To_English/Translate_Chinese_To_English.txt")
orderedDict

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/wy/p74394pn1_97m7d_x9t6_j0h0000gp/T/jieba.cache
Loading model cost 0.714 seconds.
Prefix dict has been built successfully.


OrderedDict([(0,
              Sentence(chineseWords=('海湾', '里', '平静', '海水', '的', '迷人', '景色'), equivalentEnglish=('A', 'magical', 'view', 'over', 'the', 'calm', 'waters', 'of', 'the', 'bay'))),
             (1,
              Sentence(chineseWords=('计算机病毒', '的', '一种', '主要', '特性'), equivalentEnglish=('A', 'main', 'feature', 'of', 'computer', 'virus'))),
             (2,
              Sentence(chineseWords=('一种', '致命', '的', '疾病'), equivalentEnglish=('A', 'malignant', 'disease'))),
             (3,
              Sentence(chineseWords=('人生', '只有', '一回', '死'), equivalentEnglish=('A', 'man', 'can', 'die', 'but', 'once'))),
             (4,
              Sentence(chineseWords=('出去', '吃', '晚餐', '如何'), equivalentEnglish=('How', 'about', 'going', 'out', 'for', 'dinner'))),
             (5,
              Sentence(chineseWords=('光阴', '一去不复返'), equivalentEnglish=('Time', 'past', 'can', 'not', 'be', 'called', 'back', 'again'))),
             (6,
              Sentence(chineseWords=('时间', '检验', '真理'),

In [17]:
def read_data_through_each_lines(filename):
    """Read tagged sentence data"""
    with open(filename, 'r') as f:
        sentence_lines = [l.split("\n") for l in f.read().split("\n\n")]
    return OrderedDict(((s[0], Sentence(*zip(*[l.strip().split("\t")
                        for l in s[1:]]))) for s in sentence_lines if s[0]))

def read_tags(filename):
    """Read a list of word tag classes"""
    with open(filename, 'r') as f:
        tags = f.read().split("\n")
    return frozenset(tags)

Sentence = namedtuple("Sentence", "chineseWords equivalentEnglish")

In [18]:
sentences = read_data_through_each_lines("Translate_Chinese_To_English/Translate_Chinese_To_English_simple.txt")
print(sentences)

OrderedDict([('1', Sentence(chineseWords=('一个', '国内', '战争', '迷', '。'), equivalentEnglish=('a', 'Civil', 'War', 'buff', '.'))), ('2', Sentence(chineseWords=('一个', '法国的', '庄园', '建筑', '。'), equivalentEnglish=('a', 'French', 'manor', 'house', '.'))), ('3', Sentence(chineseWords=('一条', '坏', '新闻', '。'), equivalentEnglish=('a', 'bad', 'news', '.'))), ('4', Sentence(chineseWords=('一个', '漂亮的', '角球', '。'), equivalentEnglish=('a', 'beautiful', 'corner-kick', '.'))), ('5', Sentence(chineseWords=('我', '想', '吃', '东西', '。'), equivalentEnglish=('i', 'want', 'eat', 'something', '.'))), ('6', Sentence(chineseWords=('一个', '火车', '员工', '管理', '一条', '火车', '。'), equivalentEnglish=('a', 'train', 'employee', 'in charge of', 'a', 'train', '.'))), ('7', Sentence(chineseWords=('我', '有', '很多', '问题', '。'), equivalentEnglish=('i', 'have', 'a lot of', 'questions', '.'))), ('8', Sentence(chineseWords=('一', '大群', '抗议者', '聚集', '一起', '。'), equivalentEnglish=('A', 'large group of', 'protesters', 'gathered', 'together', '.'

In [19]:
class Dataset(namedtuple("_Dataset", "sentences keys vocab X Y training_set testing_set N stream")):
    def __new__(cls, datafile, train_test_split=0.8, seed=112890):
        ### tagset = read_tags(tagfile)
        sentences = read_data_through_each_lines(datafile)
        print(sentences)
        keys = tuple(sentences.keys())
        wordset = frozenset(chain(*[s.chineseWords for s in sentences.values()]))
        word_sequences = tuple([sentences[k].chineseWords for k in keys])
        tag_sequences = tuple([sentences[k].equivalentEnglish for k in keys])
        N = sum(1 for _ in chain(*(s.chineseWords for s in sentences.values())))
        
        # split data into train/test sets
        _keys = list(keys)
        if seed is not None: random.seed(seed)
        random.shuffle(_keys)
        split = int(train_test_split * len(_keys))
        training_data = Subset(sentences, _keys[:split])
        testing_data = Subset(sentences, _keys[split:])
        stream = tuple(zip(chain(*word_sequences), chain(*tag_sequences)))
        return super().__new__(cls, dict(sentences), keys, wordset, word_sequences,
                               tag_sequences, training_data, testing_data, N, stream.__iter__)

    def __len__(self):
        return len(self.sentences)

    def __iter__(self):
        return iter(self.sentences.items())
    
    
class Subset(namedtuple("BaseSet", "sentences keys vocab X tagset Y N stream")):
    def __new__(cls, sentences, keys):
        word_sequences = tuple([sentences[k].chineseWords for k in keys])
        tag_sequences = tuple([sentences[k].equivalentEnglish for k in keys])
        wordset = frozenset(chain(*word_sequences))
        tagset = frozenset(chain(*tag_sequences))
        N = sum(1 for _ in chain(*(sentences[k].chineseWords for k in keys)))
        stream = tuple(zip(chain(*word_sequences), chain(*tag_sequences)))
        return super().__new__(cls, {k: sentences[k] for k in keys}, keys, wordset, word_sequences,
                               tagset, tag_sequences, N, stream.__iter__)

    def __len__(self):
        return len(self.sentences)

    def __iter__(self):
        return iter(self.sentences.items())

In [20]:
data = Dataset("Translate_Chinese_To_English/Translate_Chinese_To_English_simple.txt", train_test_split=0.8)

OrderedDict([('1', Sentence(chineseWords=('一个', '国内', '战争', '迷', '。'), equivalentEnglish=('a', 'Civil', 'War', 'buff', '.'))), ('2', Sentence(chineseWords=('一个', '法国的', '庄园', '建筑', '。'), equivalentEnglish=('a', 'French', 'manor', 'house', '.'))), ('3', Sentence(chineseWords=('一条', '坏', '新闻', '。'), equivalentEnglish=('a', 'bad', 'news', '.'))), ('4', Sentence(chineseWords=('一个', '漂亮的', '角球', '。'), equivalentEnglish=('a', 'beautiful', 'corner-kick', '.'))), ('5', Sentence(chineseWords=('我', '想', '吃', '东西', '。'), equivalentEnglish=('i', 'want', 'eat', 'something', '.'))), ('6', Sentence(chineseWords=('一个', '火车', '员工', '管理', '一条', '火车', '。'), equivalentEnglish=('a', 'train', 'employee', 'in charge of', 'a', 'train', '.'))), ('7', Sentence(chineseWords=('我', '有', '很多', '问题', '。'), equivalentEnglish=('i', 'have', 'a lot of', 'questions', '.'))), ('8', Sentence(chineseWords=('一', '大群', '抗议者', '聚集', '一起', '。'), equivalentEnglish=('A', 'large group of', 'protesters', 'gathered', 'together', '.'

In [21]:
print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))

assert len(data) == len(data.training_set) + len(data.testing_set), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"

There are 65 sentences in the corpus.
There are 52 sentences in the training set.
There are 13 sentences in the testing set.


In [22]:
key = '1'
print("Sentence: {}".format(key))
print("chineseWords:\n\t{!s}".format(data.sentences[key].chineseWords))
print("equivalentEnglish:\n\t{!s}".format(data.sentences[key].equivalentEnglish))

Sentence: 1
chineseWords:
	('一个', '国内', '战争', '迷', '。')
equivalentEnglish:
	('a', 'Civil', 'War', 'buff', '.')


In [23]:
# accessing words with Dataset.X and tags with Dataset.Y 
for i in range(2):    
    print("Sentence {}:".format(i + 1), data.X[i])
    print()
    print("English Labels {}:".format(i + 1), data.Y[i])
    print()

Sentence 1: ('一个', '国内', '战争', '迷', '。')

English Labels 1: ('a', 'Civil', 'War', 'buff', '.')

Sentence 2: ('一个', '法国的', '庄园', '建筑', '。')

English Labels 2: ('a', 'French', 'manor', 'house', '.')



In [24]:
print("\nStream (word, tag) pairs:\n")
for i, pair in enumerate(data.stream()):
    print("\t", pair)
    if i > 20: break


Stream (word, tag) pairs:

	 ('一个', 'a')
	 ('国内', 'Civil')
	 ('战争', 'War')
	 ('迷', 'buff')
	 ('。', '.')
	 ('一个', 'a')
	 ('法国的', 'French')
	 ('庄园', 'manor')
	 ('建筑', 'house')
	 ('。', '.')
	 ('一条', 'a')
	 ('坏', 'bad')
	 ('新闻', 'news')
	 ('。', '.')
	 ('一个', 'a')
	 ('漂亮的', 'beautiful')
	 ('角球', 'corner-kick')
	 ('。', '.')
	 ('我', 'i')
	 ('想', 'want')
	 ('吃', 'eat')
	 ('东西', 'something')


In [25]:
chineseWords = [chineseWords for i, (chineseWords, equivalentEnglish) in enumerate(data.training_set.stream())]
equivalentEnglish = [equivalentEnglish for i, (chineseWords, equivalentEnglish) in enumerate(data.training_set.stream())]
index = [i for i, (chineseWords, equivalentEnglish) in enumerate(data.training_set.stream())]
chineseWords[0:4], equivalentEnglish[0:4], index[0:4]

(['我们的', '班', '有', '一个'], ['our', 'class', 'have', 'a'], [0, 1, 2, 3])

# MFC Model

In [26]:
def pair_counts(chineseWords, equivalentEnglish):
    d = defaultdict(lambda: defaultdict(int))
    for chineseWords, equivalentEnglish in zip(chineseWords, equivalentEnglish):
        d[chineseWords][equivalentEnglish] += 1
    return d
        
word_counts = pair_counts(chineseWords, equivalentEnglish)

In [27]:
mfc_table = dict((chineseWords, max(equivalentEnglish.keys(), key=lambda key: equivalentEnglish[key])) 
                 for chineseWords, equivalentEnglish in word_counts.items())

In [28]:
i = 0
for key, value in mfc_table.items():
    print(key, value)
    i += 1
    if i > 5: break

我们的 our
班 class
有 have
一个 a
很 very
优秀的 good


In [31]:
FakeState = namedtuple('FakeState', 'name')

class MFCTagger:
    missing = FakeState(name = '<MISSING>')
    
    def __init__(self, table):
        self.table = defaultdict(lambda: MFCTagger.missing)
        self.table.update({chineseWords: FakeState(name=equivalentEnglish) for chineseWords, equivalentEnglish in table.items()})
        
    def viterbi(self, seq):
        """This method simplifies predictions by matching the Pomegranate viterbi() interface"""
        return 0., list(enumerate(["<start>"] + [self.table[w] for w in seq] + ["<end>"]))

In [32]:
mfc_model = MFCTagger(mfc_table)

In [33]:
def replace_unknown(sequence):
    return [w if w in data.training_set.vocab else 'nan' for w in sequence]

def simplify_decoding(X, model):    
    _, state_path = model.viterbi(replace_unknown(X))
    return [state[1].name for state in state_path[1:-1]]

In [34]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Sentence: {}\n".format(data.sentences[key].chineseWords))
    print("Predicted English:\n-----------------")
    print(simplify_decoding(data.sentences[key].chineseWords, mfc_model))
    print()
    print("Actual English:\n--------------")
    print(data.sentences[key].equivalentEnglish)
    print("\n")

Sentence Key: 41

Sentence: ('终于', '打扫', '完了', '。')

Predicted English:
-----------------
['<MISSING>', '<MISSING>', '<MISSING>', '.']

Actual English:
--------------
('finally', 'clean', 'finish', '.')


Sentence Key: 37

Sentence: ('摆', '小桌子', '、')

Predicted English:
-----------------
['<MISSING>', '<MISSING>', ',']

Actual English:
--------------
('place', 'table', ',')


Sentence Key: 26

Sentence: ('一窝蜂的', '都', '出去', '，')

Predicted English:
-----------------
['<MISSING>', '<MISSING>', 'go out', ',']

Actual English:
--------------
('a lot', 'all', 'get out', ',')




In [35]:
def accuracy(X, Y, model):
    
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        try:
            most_likely_tags = simplify_decoding(observations, model)
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

In [36]:
mfc_training_acc = accuracy(data.training_set.X, data.training_set.Y, mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))

mfc_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, mfc_model)
print("testing accuracy mfc_model: {:.2f}%".format(100 * mfc_testing_acc))

training accuracy mfc_model: 95.42%
testing accuracy mfc_model: 45.33%


In [37]:
def translateTest(chineseWord):
    result = ''
    words = simplify_decoding(chineseWord, mfc_model)
    for word in words:
        result += word
    return result

In [38]:
translateTest('这次作业好难啊')

'this<MISSING><MISSING><MISSING>good<MISSING><MISSING>'

# Hidden Markov Model

In [39]:
def unigram_counts(sequences):
    return Counter(sequences)

equivalentEnglishs = [equivalentEnglish for i, (chineseWords, equivalentEnglish) in enumerate(data.training_set.stream())]
tag_unigrams = unigram_counts(equivalentEnglishs)
tag_unigrams

Counter({'our': 6,
         'class': 8,
         'have': 2,
         'a': 15,
         'very': 5,
         'good': 5,
         'classmate': 1,
         ',': 27,
         'yesterday': 1,
         'after school': 1,
         'time': 1,
         'not only': 2,
         'want': 1,
         'well': 1,
         'study': 3,
         'annual': 1,
         'student': 4,
         'and': 2,
         'in': 3,
         'is': 6,
         'usually': 1,
         'been': 1,
         'Praise': 1,
         'character': 1,
         'Civil': 1,
         'War': 1,
         'buff': 1,
         '.': 26,
         'xiayu': 1,
         'place': 1,
         'garbage': 2,
         'bag': 1,
         'more': 1,
         'care': 1,
         'collective': 3,
         'bad': 1,
         'news': 1,
         'I': 10,
         'eat': 1,
         'one': 1,
         'apple': 1,
         'knead': 1,
         'knee': 1,
         'she': 7,
         'also': 3,
         'like': 1,
         'late': 2,
         'breakfast': 1,
  

In [40]:
def bigram_counts(sequences):
    return Counter(sequences)

equivalentEnglishs = [equivalentEnglish for i, (chineseWords, equivalentEnglish) in enumerate(data.stream())]
o = [(equivalentEnglish[i],equivalentEnglish[i+1]) for i in range(0,len(equivalentEnglish)-2,2)]
equivalentEnglish_bigrams = bigram_counts(o)
equivalentEnglish_bigrams

Counter({('our', 'class'): 3,
         ('have', 'a'): 1,
         ('very', 'good'): 1,
         ('classmate', ','): 1,
         ('yesterday', 'after school'): 1,
         ('time', ','): 1,
         ('not only', 'want'): 1,
         ('well', 'study'): 1,
         (',', 'annual'): 1,
         ('good', 'student'): 3,
         (',', 'and'): 1,
         ('in', 'our'): 1,
         ('class', 'is'): 1,
         ('usually', 'been'): 1,
         ('Praise', 'character'): 1,
         (',', 'a'): 7,
         ('Civil', 'War'): 1,
         ('buff', '.'): 1,
         ('xiayu', 'place'): 1,
         ('garbage', 'in'): 1,
         ('a', 'bag'): 1,
         (',', 'more'): 1,
         ('care', 'our'): 1,
         ('class', 'collective'): 2,
         ('bad', 'news'): 1,
         ('.', 'I'): 3,
         ('eat', 'one'): 1,
         ('apple', '.'): 1,
         ('knead', 'knee'): 1,
         ('she', 'also'): 1,
         ('very', 'like'): 1,
         ('.', 'a'): 2,
         ('late', 'breakfast'): 1,
         ('

In [41]:
def starting_counts(sequences):
    return Counter(sequences)

equivalentEnglishs = [equivalentEnglish for i, (chineseWords, equivalentEnglish) in enumerate(data.stream())]
starts_English = [i[0] for i in data.Y]
equivalentEnglish_starts = starting_counts(starts_English)
equivalentEnglish_starts

Counter({'a': 10,
         'i': 2,
         'A': 1,
         'a line of': 1,
         'Please': 1,
         'last night': 1,
         'our': 1,
         'she': 7,
         'and': 1,
         'annual': 1,
         'yesterday': 1,
         'student': 1,
         'a lot': 1,
         'I': 9,
         'see': 1,
         'ground': 1,
         'my': 1,
         'think': 1,
         'not too late': 1,
         'we': 2,
         'with': 2,
         'place': 1,
         'wipe': 1,
         'very': 1,
         'after': 1,
         'finally': 1,
         'xiayu': 1,
         'close': 1,
         'go': 1,
         'knead': 1,
         'insist': 1,
         'this': 1,
         'not only': 1,
         'more': 1,
         'because': 1,
         'march': 1,
         'whole body': 1,
         'one day': 1,
         'after that': 1})

In [42]:
def ending_counts(sequences):    
    return Counter(sequences)

end_English = [i[len(i)-1] for i in data.Y]
English_ends = ending_counts(end_English)
English_ends

Counter({'.': 32, ',': 31, 'busy': 1, 'knee': 1})

In [43]:
end_English = [i[len(i)-2] for i in data.Y]
English_ends = ending_counts(end_English)
English_ends

Counter({'buff': 1,
         'house': 1,
         'news': 1,
         'corner-kick': 1,
         'something': 2,
         'train': 1,
         'questions': 1,
         'together': 1,
         'waterfall': 1,
         'meeting': 1,
         'loafer': 1,
         'vehicles': 1,
         'bread': 1,
         'aunt': 1,
         'rain': 1,
         'trick': 1,
         'classmate': 1,
         'first-class': 1,
         'character': 1,
         'xiayu': 1,
         'student': 3,
         'collective': 2,
         'time': 1,
         'ring bell': 1,
         'get out': 1,
         'pace': 1,
         'eye': 1,
         'massly': 1,
         'paper': 1,
         'after': 2,
         'play': 1,
         'tidy': 1,
         'classroom': 1,
         'clean': 1,
         'table': 1,
         'blackboard': 1,
         'very': 1,
         'a while': 1,
         'finish': 1,
         'bag': 1,
         'class': 1,
         'door': 1,
         'garbage': 1,
         'fell': 1,
         'up': 1,
    

In [44]:
hmm_model = HiddenMarkovModel(name="base-hmm-tagger")

someEquivalentEnglish = [equivalentEnglish for i, (chineseWord, equivalentEnglish) in enumerate(data.stream())]
chineseWords = [chineseWord for i, (chineseWord, equivalentEnglish) in enumerate(data.stream())]

someEquivalentEnglish_count = unigram_counts(someEquivalentEnglish)
equivalentEnglish_chineseWords_count = pair_counts(someEquivalentEnglish, chineseWords)

starting_equivalentEnglish_list = [i[0] for i in data.Y]
ending_equivalentEnglish_list = [i[len(i)-2] for i in data.Y]

starting_equivalentEnglish_count = starting_counts(starting_equivalentEnglish_list) #the number of times a equivalentEnglish occured at the start
ending_equivalentEnglish_count = ending_counts(ending_equivalentEnglish_list)       #the number of times a equivalentEnglish occured at the end

equivalentEnglish_chineseWords_count

defaultdict(<function __main__.pair_counts.<locals>.<lambda>()>,
            {'a': defaultdict(int, {'一个': 14, '一条': 2}),
             'Civil': defaultdict(int, {'国内': 1}),
             'War': defaultdict(int, {'战争': 1}),
             'buff': defaultdict(int, {'迷': 1}),
             '.': defaultdict(int, {'。': 32}),
             'French': defaultdict(int, {'法国的': 1}),
             'manor': defaultdict(int, {'庄园': 1}),
             'house': defaultdict(int, {'建筑': 1}),
             'bad': defaultdict(int, {'坏': 1}),
             'news': defaultdict(int, {'新闻': 1}),
             'beautiful': defaultdict(int, {'漂亮的': 1}),
             'corner-kick': defaultdict(int, {'角球': 1}),
             'i': defaultdict(int, {'我': 2}),
             'want': defaultdict(int, {'想': 1, '要': 1}),
             'eat': defaultdict(int, {'吃': 2}),
             'something': defaultdict(int, {'东西': 2}),
             'train': defaultdict(int, {'火车': 2}),
             'employee': defaultdict(int, {'员工': 1}),
     

In [45]:
ending_equivalentEnglish_list

['buff',
 'house',
 'news',
 'corner-kick',
 'something',
 'train',
 'questions',
 'together',
 'waterfall',
 'meeting',
 'loafer',
 'vehicles',
 'bread',
 'aunt',
 'rain',
 'trick',
 'classmate',
 'first-class',
 'character',
 'xiayu',
 'student',
 'student',
 'collective',
 'time',
 'ring bell',
 'get out',
 'pace',
 'eye',
 'massly',
 'paper',
 'after',
 'play',
 'tidy',
 'after',
 'classroom',
 'clean',
 'table',
 'blackboard',
 'very',
 'a while',
 'finish',
 'bag',
 'class',
 'door',
 'garbage',
 'fell',
 'up',
 'knead',
 'home',
 'student',
 'study',
 'study',
 'collective',
 'home',
 'side',
 'walking',
 'walking',
 'something',
 'know',
 'shopping',
 'uncle',
 'apple',
 'pears',
 'pears',
 'apples']

Let's convert word frequencies by POS tag to probabilities by dividing by the total number of words per POS tag, yielding the distribution of words.

We'll define HMM emission probabilities using that distribution.

In [55]:
to_pass_states = []
for equivalentEnglish, chineseWords_dict in equivalentEnglish_chineseWords_count.items():
    total = float(sum(chineseWords_dict.values()))
    distribution = {chineseWords: count/total for chineseWords, count in chineseWords_dict.items()}
    equivalentEnglish_emissions = DiscreteDistribution(distribution)
    equivalentEnglish_state = State(equivalentEnglish_emissions, name=equivalentEnglish)
    to_pass_states.append(equivalentEnglish_state)
    print(distribution)



{'一个': 0.875, '一条': 0.125}
{'国内': 1.0}
{'战争': 1.0}
{'迷': 1.0}
{'。': 1.0}
{'法国的': 1.0}
{'庄园': 1.0}
{'建筑': 1.0}
{'坏': 1.0}
{'新闻': 1.0}
{'漂亮的': 1.0}
{'角球': 1.0}
{'我': 1.0}
{'想': 0.5, '要': 0.5}
{'吃': 1.0}
{'东西': 1.0}
{'火车': 1.0}
{'员工': 1.0}
{'管理': 1.0}
{'有': 1.0}
{'很多': 1.0}
{'问题': 1.0}
{'一': 1.0}
{'大群': 1.0}
{'抗议者': 1.0}
{'聚集': 1.0}
{'一起': 1.0}
{'大瀑布': 1.0}
{'或': 1.0}
{'高': 1.0}
{'瀑布': 1.0}
{'推迟的': 0.5, '推迟了的': 0.5}
{'早餐': 1.0}
{';': 0.5, '；': 0.5}
{'会议': 1.0}
{'懒惰': 1.0}
{'闲散': 1.0}
{'的人': 1.0}
{'游手好闲者': 1.0}
{'一行': 1.0}
{'等待': 1.0}
{'人': 1.0}
{'车辆': 1.0}
{'请': 1.0}
{'给': 1.0}
{'我': 1.0}
{'切片': 1.0}
{'面包': 1.0}
{'长寿的': 1.0}
{'姑姑': 1.0}
{'昨晚': 1.0}
{'苏格兰': 1.0}
{'下了': 1.0}
{'许多': 1.0}
{'雨': 1.0}
{'可鄙的': 1.0}
{'恶作剧': 1.0}
{'我们的': 0.5, '我们': 0.5}
{'班': 0.5, '班的': 0.3, '教室': 0.2}
{'很': 0.4, '特别': 0.2, '非常': 0.4}
{'优秀的': 0.4, '三好': 0.2, '好': 0.2, '优秀': 0.2}
{'同学': 1.0}
{'，': 0.918918918918919, '、': 0.05405405405405406, ',': 0.02702702702702703}
{'她': 1.0}
{'不光': 0.5, '不仅': 0.5}
{'学习': 1.0}
{'

In [56]:
distribution

{'苹果': 1.0}

In [57]:
distribution['苹果']

1.0

`to_pass_states` yields the probability distribution of words per POS tag:

In [58]:
to_pass_states

[{
     "class" : "State",
     "distribution" : {
         "class" : "Distribution",
         "dtype" : "str",
         "name" : "DiscreteDistribution",
         "parameters" : [
             {
                 "\u4e00\u4e2a" : 0.875,
                 "\u4e00\u6761" : 0.125
             }
         ],
         "frozen" : false
     },
     "name" : "a",
     "weight" : 1.0
 }, {
     "class" : "State",
     "distribution" : {
         "class" : "Distribution",
         "dtype" : "str",
         "name" : "DiscreteDistribution",
         "parameters" : [
             {
                 "\u56fd\u5185" : 1.0
             }
         ],
         "frozen" : false
     },
     "name" : "Civil",
     "weight" : 1.0
 }, {
     "class" : "State",
     "distribution" : {
         "class" : "Distribution",
         "dtype" : "str",
         "name" : "DiscreteDistribution",
         "parameters" : [
             {
                 "\u6218\u4e89" : 1.0
             }
         ],
         "frozen" : f

Let's add states to our model:

In [59]:
hmm_model.add_states() 

The start probability for each tag is how many times it is a sentence-starting POS tag divided by its total count. We build the starting transitions for our HMM model:

In [60]:
start_prob={}

for equivalentEnglish in someEquivalentEnglish:
    start_prob[equivalentEnglish] = starting_equivalentEnglish_count[equivalentEnglish] / someEquivalentEnglish_count[equivalentEnglish]

for equivalentEnglish_state in to_pass_states :
    hmm_model.add_transition(hmm_model.start, equivalentEnglish_state, start_prob[equivalentEnglish_state.name])  

The end probability for each tag is how many times it is a sentence-ending POS tag divided by its total count. We build the ending transitions for our HMM model:

In [61]:
end_prob={}

for equivalentEnglish in someEquivalentEnglish:
    end_prob[equivalentEnglish] = ending_equivalentEnglish_count[equivalentEnglish]/someEquivalentEnglish_count[equivalentEnglish]
    
for someEquivalentEnglish_state in to_pass_states :
    hmm_model.add_transition(equivalentEnglish_state, hmm_model.end, end_prob[equivalentEnglish_state.name])

We now add the transition probabilities for our model, which uses our POS bigrams to enumerate what the probabilities are for transiting from one POS tag to another.

In [62]:
transition_prob_pair={}

for key in equivalentEnglish_bigrams.keys():
    transition_prob_pair[key] = equivalentEnglish_bigrams.get(key)/someEquivalentEnglish_count[key[0]]
    
    
for equivalentEnglish_state in to_pass_states :
    for next_equivalentEnglish_state in to_pass_states:
        if (equivalentEnglish_state.name, next_equivalentEnglish_state.name) not in transition_prob_pair:
            transition_prob_pair[(equivalentEnglish_state.name, next_equivalentEnglish_state.name)]=0
        hmm_model.add_transition(equivalentEnglish_state, next_equivalentEnglish_state, transition_prob_pair[(equivalentEnglish_state.name, next_equivalentEnglish_state.name)])
       

We *bake* our model:

In [63]:
hmm_model.bake()

We can now evaluate the accuracy of our HMM model and compare it to our BOW model:

In [64]:
hmm_training_acc = accuracy(data.training_set.X, data.training_set.Y, hmm_model)
print("training accuracy basic hmm model: {:.2f}%".format(100 * hmm_training_acc))

hmm_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, hmm_model)
print("testing accuracy basic hmm model: {:.2f}%".format(100 * hmm_testing_acc))

training accuracy basic hmm model: 0.00%
testing accuracy basic hmm model: 0.00%


Here's a decoding example:

In [73]:
for key in data.testing_set.keys[:1]:
    print("Sentence Key: {}\n".format(key))
    print("Sentence: {}\n".format(data.sentences[key].chineseWords))
    print("translate: {}\n".format(data.sentences[key].equivalentEnglish))
    print("\n")

Sentence Key: 41

Sentence: ('终于', '打扫', '完了', '。')

translate: ('finally', 'clean', 'finish', '.')



